In [ ]:
pip install tabulate


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('your_dataset_path.csv')

# Filter for the 2023 draft class
df_2023 = df[df['Draft Year'] == 2023]

# Handling NaN values and reordering based on advanced stats
df_2023['WS_Reordering'] = df_2023['WS'].rank(method='max', ascending=False, na_option='keep')
df_2023['WS48_Reordering'] = df_2023['WS/48'].rank(method='max', ascending=False, na_option='keep')
df_2023['BPM_Reordering'] = df_2023['BPM'].rank(method='max', ascending=False, na_option='keep')
df_2023['VORP_Reordering'] = df_2023['VORP'].rank(method='max', ascending=False, na_option='keep')

# Preparing the final output
final_output = df_2023[['Pk', 'Tm', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].copy()
final_output.rename(columns={'Pk': 'Pick', 'Tm': 'Team'}, inplace=True)
final_output = final_output.sort_values('Pick').reset_index(drop=True)

# Export the final output to CSV
final_output.to_csv('2023_NBA_Draft_Reordering_Full.csv', index=False)


In [ ]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from tabulate import tabulate

# Load the dataset
df = pd.read_csv('/content/NBA Draft Data (2021-2023 ALL PICKS).csv')

# Filter for the 2023 draft class
df_2023 = df[df['Draft Year'] == 2023]

# Handling NaN values and reordering based on advanced stats
df_2023['WS_Reordering'] = df_2023['WS'].rank(method='max', ascending=False, na_option='keep')
df_2023['WS48_Reordering'] = df_2023['WS/48'].rank(method='max', ascending=False, na_option='keep')
df_2023['BPM_Reordering'] = df_2023['BPM'].rank(method='max', ascending=False, na_option='keep')
df_2023['VORP_Reordering'] = df_2023['VORP'].rank(method='max', ascending=False, na_option='keep')

# Correcting 'Average Rank' calculation
df_2023['Average_Rank'] = df_2023[['WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].mean(axis=1)

# Preparing the final output
final_output = df_2023[['Pk', 'Tm', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering', 'Average_Rank']].copy()
final_output.rename(columns={'Pk': 'Pick', 'Tm': 'Team'}, inplace=True)
final_output = final_output.sort_values('Pick').reset_index(drop=True)

# Attempting Spearman rank correlation calculation again, with NaN handling
correlations = [
    spearmanr(df_2023['Pick'], df_2023['WS_Reordering'].fillna(-100)).correlation,
    spearmanr(df_2023['Pick'], df_2023['WS48_Reordering'].fillna(-100)).correlation,
    spearmanr(df_2023['Pick'], df_2023['BPM_Reordering'].fillna(-100)).correlation,
    spearmanr(df_2023['Pick'], df_2023['VORP_Reordering'].fillna(-100)).correlation
]
average_spearman = np.nanmean(correlations)  # Using nanmean to ignore NaNs

top_10_display = final_output_with_avg.head(10)

# Converting the relevant part of the DataFrame to a list of lists for tabulate
table_data = top_10_display[['Pick', 'Team', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering', 'Average_Rank']].values.tolist()

# Printing the table using tabulate
print(tabulate(table_data, headers=['Pick', 'Team', 'Player', 'WS Reordering', 'WS/48 Reordering', 'BPM Reordering', 'VORP Reordering', 'Average Rank'], tablefmt='pretty'))

# Export the final output to CSV
#output_path = '2023_NBA_Draft_Reordering_Full_with_Avg.csv'
#final_output.to_csv(output_path, index=False)

print(f"Average Spearman Rank Correlation: {average_spearman}")

In [ ]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from tabulate import tabulate

# Load the dataset
df = pd.read_csv('/content/2023 Draft CSV8.csv', encoding='utf-8')
df.fillna(-100, inplace=True)

# Filter for the 2023 draft class and make a copy to avoid SettingWithCopyWarning
df_2023 = df[df['Draft Year'] == 2023].copy()

# Handling NaN values and reordering based on advanced stats
df_2023['WS_Reordering'] = df_2023['WS'].rank(method='max', ascending=False, na_option='keep')
df_2023['WS48_Reordering'] = df_2023['WS/48'].rank(method='max', ascending=False, na_option='keep')
df_2023['BPM_Reordering'] = df_2023['BPM'].rank(method='max', ascending=False, na_option='keep')
df_2023['VORP_Reordering'] = df_2023['VORP'].rank(method='max', ascending=False, na_option='keep')

# Calculate 'Average Rank'
df_2023['Average_Rank'] = df_2023[['WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].mean(axis=1)
print(df_2023['Average_Rank'])

# Rename columns to ensure we have 'Pick' and 'Team' instead of 'Pk' and 'Tm'
final_output = df_2023.rename(columns={'Pk': 'Pick', 'Tm': 'Team'})

# Sorting by 'Pick' for a coherent view
final_output = final_output.sort_values('Pick').reset_index(drop=True)

# Calculate the Spearman rank correlations to see how well the draft order matches the reordered rankings
correlations = [
    spearmanr(final_output['Pick'], final_output['WS_Reordering'].fillna(-100)).correlation,
    spearmanr(final_output['Pick'], final_output['WS48_Reordering'].fillna(-100)).correlation,
    spearmanr(final_output['Pick'], final_output['BPM_Reordering'].fillna(-100)).correlation,
    spearmanr(final_output['Pick'], final_output['VORP_Reordering'].fillna(-100)).correlation
]
print(correlations)
spearman_corr_avg_rank = spearmanr(final_output['Pick'], final_output['Average_Rank']).correlation
print(spearman_corr_avg_rank)
average_spearman = np.nanmean(correlations)  # Using nanmean to handle potential NaNs in correlations

# Displaying the top 10 formatted table
top_10_display = final_output.head(10)
table_data = top_10_display[['Pick', 'Team', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering', 'Average_Rank']].values.tolist()

# Printing the table using tabulate
print(tabulate(table_data, headers=['Pick', 'Team', 'Player', 'WS Reordering', 'WS/48 Reordering', 'BPM Reordering', 'VORP Reordering', 'Average Rank'], tablefmt='pretty'))

# Print the average Spearman rank correlation
print(f"Average Spearman Rank Correlation: {average_spearman}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import spearmanr
from tabulate import tabulate

# Load the dataset
df = pd.read_csv('/content/Testing DRAFT CLASSES.csv', encoding='utf-8')
df.fillna(-100, inplace=True)

# Create an empty list to store results
results = []

# Loop through each year from 1984 to 2023
for year in range(1984, 2024):
    # Filter for the specific draft year and make a copy
    df_year = df[df['Draft Year'] == year].copy()

    # Handling NaN values and reordering based on advanced stats
    df_year['WS_Reordering'] = df_year['WS'].rank(method='max', ascending=False, na_option='keep')
    df_year['WS48_Reordering'] = df_year['WS/48'].rank(method='max', ascending=False, na_option='keep')
    df_year['BPM_Reordering'] = df_year['BPM'].rank(method='max', ascending=False, na_option='keep')
    df_year['VORP_Reordering'] = df_year['VORP'].rank(method='max', ascending=False, na_option='keep')

    # Calculate 'Average Rank'
    df_year['Average_Rank'] = df_year[['WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].mean(axis=1)

    # Rename columns to ensure we have 'Pick' and 'Team' instead of 'Pk' and 'Tm'
    final_output = df_year.rename(columns={'Pk': 'Pick', 'Tm': 'Team'})

    # Calculate the Spearman rank correlations to see how well the draft order matches the reordered rankings
    correlations = [
        spearmanr(final_output['Pick'], final_output['WS_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['WS48_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['BPM_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['VORP_Reordering'].fillna(-100)).correlation
    ]
    spearman_corr_avg_rank = spearmanr(final_output['Pick'], final_output['Average_Rank']).correlation
    average_spearman = np.nanmean(correlations)  # Using nanmean to handle potential NaNs in correlations

    # Append results to the list
    results.append([year, average_spearman, spearman_corr_avg_rank])

# Displaying the results in a formatted table
results_df = pd.DataFrame(results, columns=['Year', 'Average Spearman Correlation', 'Spearman Correlation (Original vs. Average Rank)'])

# Assign colors based on Spearman correlation values
def get_color(value):
    # Define colormap ranging from white to light blue
    cmap = plt.cm.get_cmap('Blues')
    norm = mcolors.Normalize(-1, 1)  # Normalize values to range from -1 to 1
    return mcolors.to_hex(cmap(norm(value)))

# Apply color formatting to the results DataFrame
results_df['Average Spearman Color'] = results_df['Average Spearman Correlation'].apply(get_color)
results_df['Spearman Correlation Color'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(get_color)

# Truncate numerical values to 3 decimal places with zero-padding
results_df['Average Spearman Correlation'] = results_df['Average Spearman Correlation'].apply(lambda x: f"{x:.3f}")
results_df['Spearman Correlation (Original vs. Average Rank)'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(lambda x: f"{x:.3f}")

# Save results to a CSV file
results_df.to_csv('spearman_correlation_results_with_colors.csv', index=False)

# Print the tabulated results with color values
print(tabulate(results_df[['Year', 'Average Spearman Correlation', 'Average Spearman Color', 'Spearman Correlation (Original vs. Average Rank)', 'Spearman Correlation Color']], headers='keys', tablefmt='pretty'))


+----+------+------------------------------+------------------------+--------------------------------------------------+----------------------------+
|    | Year | Average Spearman Correlation | Average Spearman Color | Spearman Correlation (Original vs. Average Rank) | Spearman Correlation Color |
+----+------+------------------------------+------------------------+--------------------------------------------------+----------------------------+
| 0  | 1984 |            0.571             |        #1967ad         |                      0.576                       |          #1967ad           |
| 1  | 1985 |            0.332             |        #3787c0         |                      0.350                       |          #3585bf           |
| 2  | 1986 |            0.307             |        #3a8ac2         |                      0.322                       |          #3888c1           |
| 3  | 1987 |            0.630             |        #1460a8         |                      0.636    

<ipython-input-16-d0a65557df89>:52: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('Blues')
<ipython-input-16-d0a65557df89>:52: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('Blues')


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('/content/Spearman Coefficients with Colors (1984 to 2023).csv')

# Start the LaTeX table code
latex_code = "\\begin{tabular}{|l|c|c|}\n\\hline\n"
latex_code += "\\rowcolor{gray!20}\n\\textbf{Year} & \\textbf{Average Spearman Corr.} & \\textbf{Spearman Corr. (Avg. Rank)} \\\\ \\hline\n"

# Loop through the DataFrame and add rows to the LaTeX code
for index, row in df.iterrows():
    year = int(row['Year'])  # Convert year to integer
    avg_corr = row['Average Spearman Correlation']
    rank_corr = row['Spearman Correlation (Average Rank)']
    # Check the values to determine the color of the cell
    avg_corr_color = 'lightgreen' if avg_corr >= 0.5 else 'lightred'
    rank_corr_color = 'lightgreen' if rank_corr >= 0.5 else 'lightred'
    # Add the row to the LaTeX code
    latex_code += f"\\textbf{{{year}}} & \\cellcolor{{{avg_corr_color}}}{avg_corr:.3f} & \\cellcolor{{{rank_corr_color}}}{rank_corr:.3f} \\\\ \\hline\n"

# End the LaTeX table code
latex_code += "\\end{tabular}\n"

# Save the LaTeX code to a .txt file
with open('spearman_coefficients_table.txt', 'w') as file:
    file.write(latex_code)

print("LaTeX code saved as spearman_coefficients_table.txt")


LaTeX code saved as spearman_coefficients_table.txt
